In [2]:
# 1. preparation

##secured##
#my_id, my_pw here

# 2. import libraries and modules

#!pip install selenium
#!pip install pandas
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [3]:
# 3. synchronize chrome driver 

path = r"C:\Users\iwin1\Desktop\cs_portfolio\everytime_corpus\chromedriver"
driver = webdriver.Chrome(executable_path = path)

# 4. log-in and move to the board

driver.get("https://everytime.kr/login")
driver.find_element_by_name("userid").send_keys(my_id)
driver.find_element_by_name("password").send_keys(my_pw)
driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

driver.find_element_by_xpath('//*[@id="submenu"]/div/div[2]/ul/li[1]/a').click()
time.sleep(5)
driver.find_element_by_xpath('//*[@id="sheet"]/ul/li[3]/a').click()

# 5. declare a crawling def

title_data = []
content_data = []
vote_data = []
comment_data = []

def crawling(pg):
    soup = BeautifulSoup(pg,"html.parser")
    to_body = soup.find(class_="wrap articles")
    title = to_body.find_all("h2")
    content = to_body.find_all("p")
    vote = to_body.find_all(class_="vote")
    comment = to_body.find_all(class_="comment")

    for i in range(0,len(title)):
        title_data.append(title[i].text)
        content_data.append(content[i].text)
        vote_data.append(vote[i].text)
        comment_data.append(comment[i].text)

    return title_data, content_data, vote_data, comment_data

In [ ]:
# 6. start crawling 

page_num = 1
title, content, vote, comment = crawling(driver.page_source)
print("[INFO] page %d completed ... "%page_num)

while True:
    page_num += 1
    try:   
        driver.find_element_by_class_name("next").click()
        time.sleep(0.2)
        title, content, vote, comment = crawling(driver.page_source)
        print("[INFO] page %d completed ... "%page_num)

    except NoSuchElementException:
        try:
            print("[Warning] page %d cracked... re-analyzing")
            time.sleep(2)
            driver.find_element_by_class_name("next").click()
            time.sleep(2)
            title, content, vote, comment = crawling(driver.page_source)
            time.sleep(2)
            print("[INFO] page %d completed ... "%page_num)
        except:
            print("[INFO] page %d is the last page. nice work!"%page_num)
            break    

In [6]:
# 7 convert to pandas dataframe and visualize df.

every_time_dict = {"title": title, "content": content, "vote": vote, "comment": comment}
every_time_dataframe = pd.DataFrame(every_time_dict)
display(every_time_dataframe)

# 0829_added) 8. change string to int (vote)

for i in range(len(every_time_dataframe)):
    every_time_dataframe.vote[i] = int(every_time_dataframe.vote[i])
    if i%1000 == 0:
        print("[INFO] %d vote modified"%((i*100)/len(every_time_dataframe)))

#every_time_dataframe = [int(votes) for votes in every_time_dataframe.vote]

# 9 save it into csv format

every_time_dataframe.to_csv(r'C:\Users\iwin1\Desktop\cs_portfolio\everytime_corpus\final_SGU_everytime_data.csv', index=False, header = True)

,title,content,vote,comment
0,알바비 안 들어왔는데,원래 6일날 넣어주시는데 원장님 사정으로 지난 달에 8일날 주셨어 ㅠㅠ 문자 드려볼...,0,0
1,요소실,에이플 있음? 어떤식으로 평가한겨 도대체;;,0,2
2,논리학개론 ㅇㅇㅇ교수님,오늘자 파트2 쉅 올려주신 거 맞나여? 링크로 올려주신다고했는데 안올라온 것 같아서...,0,0
3,직장인 돼도 고딩같아 보인다고 들을까ㅋㅋ,작년 23살에도 19살같아 보인다고 들었는데 이젠 그래도 20초반으로는 듣고싶음,0,6
4,내가 피해자 부모였으면,술먹고 가해자 죽이는게 나을 것 같다는 생각을 하게 만들어. 법 결국 사람이 만든...,4,4
...,...,...,...,...
22903,오빠,오빠다 나도 오빠가 있으면 좋겠다.... 우리 오빠는 어디있으려나,2,4
22904,오 이제 외부인 없겠다,좋네,11,1
22905,3빠,오,0,0
22906,오 뭔가 간지난다,에베베베베베베,1,0
